In [ ]:
import pandas as pd
#import stumpy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt
import scipy
import sklearn
import copy
from tabulate import tabulate
import math
from collections import Counter

plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/'
DATASET_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/k6csv/'
Baseline_DATA_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/k6csv/Baselines/'
Aggregate_DATA_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/k6csv/Aggregates/'
Remake_DATA_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/k6csv/RngRemake/'

In [ ]:
import time

#csvName = '14456d07da6db5da228beff3ad356f824e4cd09a8d6771089eb26a862526ac9a'
#csvName = 'cd05d7b4445349ee645ea290586fd28c0c675a155eb1522485535c5c0329a908'
csvName = '36bc568292ec360f9cc2f4431fa0db953ce8714cdb1b165f4ec68ca54e7fe2be'

multiSeriesMain = pd.read_csv(DATA_PATH+csvName+'.csv')
multiSeriesMain = multiSeriesMain.loc[:, ~multiSeriesMain.columns.str.contains('^Unnamed')].astype(float).reset_index(drop=True)

In [ ]:
csvName = '13Bogo'
csvArray = pd.read_csv(Baseline_DATA_PATH+csvName+'.csv', header=None, usecols=[0,1,2,])
bogoArray = csvArray[csvArray[0].eq("get_bogo_duration_trend" )].iloc[:,1:].astype(float).reset_index(drop=True)
csvName = '13Bubble'
csvArray = pd.read_csv(Baseline_DATA_PATH+csvName+'.csv', header=None, usecols=[0,1,2,])
bubbleArray = csvArray[csvArray[0].eq("get_bubble_duration_trend" )].iloc[:,1:].astype(float).reset_index(drop=True)

In [ ]:
len(bubbleArray)

In [ ]:
startTime = int(bubbleArray[1].iloc[0:1])
endTime = int(bubbleArray[1].iloc[-1])
maxTime = round((endTime-startTime)/60)
countArray = []
for r in range(maxTime):
  countArray.append(0)
for b in range(len(bubbleArray)):
  if bubbleArray[2].iloc[b] >= SLA:
    countArray[round((startTime-bubbleArray[1].iloc[b])/60)] += 1
print(countArray)

In [ ]:
plt.plot(countArray)

In [ ]:
# csvName = '24Hour5min5Clust'
# csvName2 = False

# csvArray = pd.read_csv(Aggregate_DATA_PATH+csvName+'.csv', header=None, usecols=[0,1,2,])

# bogoArrayClust = csvArray[csvArray[0].eq("get_bogo_duration_trend" )].iloc[:,1:].astype(float).reset_index(drop=True)
# bubbleArrayClust = csvArray[csvArray[0].eq("get_bubble_duration_trend" )].iloc[:,1:].astype(float).reset_index(drop=True)
# if csvName2:
#   csvArray2 = pd.read_csv(Aggregate_DATA_PATH+csvName2+'.csv', header=None, usecols=[0,1,2,])

#   bogoArrayClust2 = csvArray2[csvArray2[0].eq("get_bogo_duration_trend" )].iloc[:,1:].astype(float).reset_index(drop=True)
#   bubbleArrayClust2 = csvArray2[csvArray2[0].eq("get_bubble_duration_trend" )].iloc[:,1:].astype(float).reset_index(drop=True)

csvName = '24HourBogo5min5Clust'
csvArray = pd.read_csv(Remake_DATA_PATH+csvName+'.csv', header=None, usecols=[0,1,2,])
bogoArrayClust = csvArray[csvArray[0].eq("get_bogo_duration_trend" )].iloc[:,1:].astype(float).reset_index(drop=True)
csvName = '24HourBubble5min5Clust'
csvArray = pd.read_csv(Remake_DATA_PATH+csvName+'.csv', header=None, usecols=[0,1,2,])
bubbleArrayClust = csvArray[csvArray[0].eq("get_bubble_duration_trend" )].iloc[:,1:].astype(float).reset_index(drop=True)

import json

restart = False
jsonName = '24hourRngRemake5min5Clust.json'
jsonData = pd.read_json('/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/k6json/'+jsonName)
if restart == False:
  clusterInfo = json.loads(jsonData['clusters'].loc['firstScenario'])
  print(len(clusterInfo))
  weights = []
  tempDict = Counter(sorted(clusterInfo))
  for c in tempDict.values():
    weights.append(c/len(clusterInfo))
  print(weights)
clusterIndices = json.loads(jsonData['indices'].loc['firstScenario'])
print(len(clusterIndices))
windowSize = 5
warmup = 5
warmupR = 5
timeR = windowSize+warmupR+1

realWeights = []
for c in clusterIndices:
  realWeights.append(len(c)/(1440-windowSize))
print(realWeights)

In [ ]:
startTime = int(bubbleArrayClust[1].iloc[0:1])
endTime = int(bubbleArrayClust[1].iloc[-1])
maxTime = round((endTime-startTime)/60)
minuteListBubble = []
for i in range(maxTime):
  minuteListBubble.append([])
for c in range(len(bubbleArrayClust)):
  tempTime = bubbleArrayClust[1].iloc[c]
  minuteTemp = round((tempTime-startTime)/60)
  minuteListBubble[minuteTemp-1].append(bubbleArrayClust[2].iloc[c])
for w in range(warmup):
  del(minuteListBubble[0])
startTime = int(bubbleArray[1].iloc[0:1])
endTime = int(bubbleArray[1].iloc[-1])
maxTime = round((endTime-startTime)/60)
minuteListBubbleReal = []
for i in range(maxTime):
  minuteListBubbleReal.append([])
for c in range(len(bubbleArray)):
  tempTime = bubbleArray[1].iloc[c]
  minuteTemp = round((tempTime-startTime)/60)
  minuteListBubbleReal[minuteTemp-1].append(bubbleArray[2].iloc[c])
for w in range(warmup):
  del(minuteListBubbleReal[0])

In [ ]:
startTime = int(bogoArrayClust[1].iloc[0:1])
endTime = int(bogoArrayClust[1].iloc[-1])
maxTime = round((endTime-startTime)/60)
minuteListBogo = []
for i in range(maxTime):
  minuteListBogo.append([])
for c in range(len(bogoArrayClust)):
  tempTime = bogoArrayClust[1].iloc[c]
  minuteTemp = round((tempTime-startTime)/60)
  minuteListBogo[minuteTemp-1].append(bogoArrayClust[2].iloc[c])
for w in range(warmup):
  del(minuteListBogo[0])
startTime = int(bogoArray[1].iloc[0:1])
endTime = int(bogoArray[1].iloc[-1])
maxTime = round((endTime-startTime)/60)
minuteListBogoReal = []
for i in range(maxTime):
  minuteListBogoReal.append([])
for c in range(len(bogoArray)):
  tempTime = bogoArray[1].iloc[c]
  minuteTemp = round((tempTime-startTime)/60)
  minuteListBogoReal[minuteTemp-1].append(bogoArray[2].iloc[c])
for w in range(warmup):
  del(minuteListBogoReal[0])

In [ ]:
sla = 20
realSlaCountBubble = []
realTransCountBubble = []
for clust in clusterIndices:
  tempCount = 0
  tempTransCount = 0
  for c in clust:
    tempTransCount += len(minuteListBubbleReal[c])
    for i in minuteListBubbleReal[c]:
      if i >= sla:
        tempCount += 1
  realTransCountBubble.append(tempTransCount)
  realSlaCountBubble.append(tempCount)
print(realSlaCountBubble)
print(realTransCountBubble)

fakeSlaCountBubble = []
fakeTransCountBubble = []
for l in range(len(clusterIndices)):
  fakeSlaCountBubble.append(0)
  fakeTransCountBubble.append(0)
iterator = 0
for clust in clusterInfo:
  tempCount = 0
  tempTransCount = 0
  for w in range(windowSize):
    for i in minuteListBubbleReal[w+(iterator*(windowSize-1))]:
      tempTransCount += 1
      if i >= sla:
        tempCount += 1
  fakeSlaCountBubble[clust] += tempCount
  fakeTransCountBubble[clust] += tempTransCount
  iterator += 1
print(fakeSlaCountBubble)
print(fakeTransCountBubble)
fakeAvgBubble = []
for i in range(len(fakeSlaCountBubble)):
  fakeAvgBubble.append(fakeSlaCountBubble[i]/fakeTransCountBubble[i])
realAvgBubble = []
for i in range(len(realSlaCountBubble)):
  realAvgBubble.append(realSlaCountBubble[i]/realTransCountBubble[i])

print(realAvgBubble)
print(fakeAvgBubble)

In [ ]:
sla = 20
realSlaCountBogo= []
realTransCountBogo = []
for clust in clusterIndices:
  tempCount = 0
  tempTransCount = 0
  for c in clust:
    tempTransCount += len(minuteListBogoReal[c])
    for i in minuteListBogoReal[c]:
      if i >= sla:
        tempCount += 1
  realTransCountBogo.append(tempTransCount)
  realSlaCountBogo.append(tempCount)
print(realSlaCountBogo)
print(realTransCountBogo)

fakeSlaCountBogo = []
fakeTransCountBogo = []
for l in range(len(clusterIndices)):
  fakeSlaCountBogo.append(0)
  fakeTransCountBogo.append(0)
iterator = 0
for clust in clusterInfo:
  tempCount = 0
  tempTransCount = 0
  for w in range(windowSize):
    for i in minuteListBogoReal[w+(iterator*(windowSize-1))]:
      tempTransCount += 1
      if i >= sla:
        tempCount += 1
  fakeSlaCountBogo[clust] += tempCount
  fakeTransCountBogo[clust] += tempTransCount
  iterator += 1
print(fakeSlaCountBogo)
print(fakeTransCountBogo)
fakeAvgBogo = []
for i in range(len(fakeSlaCountBogo)):
  fakeAvgBogo.append(fakeSlaCountBogo[i]/fakeTransCountBogo[i])
realAvgBogo = []
for i in range(len(realSlaCountBogo)):
  realAvgBogo.append(realSlaCountBogo[i]/realTransCountBogo[i])

print(realAvgBogo)
print(fakeAvgBogo)

In [ ]:
if restart:
  startTime = int(bubbleArrayClust[1].iloc[0:1])
  endTime = int(bubbleArrayClust[1].iloc[-1])
  maxTime = round((endTime-startTime)/60)
  print(maxTime)

  clustCountArray = []
  for i in range(maxTime):
    clustCountArray.append(bubbleArrayClust[2][(bubbleArrayClust[1] >= startTime+60*(i)) & (bubbleArrayClust[1] < startTime+60*(1+i)) & (SLA_Clust <= bubbleArrayClust[2])].count())
  clustAmountArray = []
  for i in range(maxTime):
    clustAmountArray.append((bubbleArrayClust[2][(bubbleArrayClust[1] >= startTime+60*(i)) & (bubbleArrayClust[1] < startTime+60*(1+i))].count()))

  realClustCountArray = []
  value = warmupR-2
  for c in range(len(clusterIndices)):
    tempArray = []
    for r in range(windowSize):
      value += 1
      tempArray.append(clustCountArray[value])
    value += warmupR+1
    realClustCountArray.append(tempArray)
  print(realClustCountArray)
  clusterSum = []
  for r in realClustCountArray:
    print(sum(r))
    clusterSum.append(sum(r))

  slaByClust = []

  for cluster in clusterIndices:
    tempSum = 0
    for c in cluster:
      tempSum += countArray[c-1]
    slaByClust.append(tempSum)

  print('clusters',clusterSum)
  print('baseline',slaByClust)

  perClusterSum = []
  for c in clusterSum:
    perClusterSum.append(c/sum(clusterSum))
  perSlaByClust = []
  for s in slaByClust:
    perSlaByClust.append(s/sum(slaByClust))
  plt.bar([1,2,3,4,5],perClusterSum)
  plt.legend(['Clusters'])
  plt.show()
  plt.bar([1,2,3,4,5],perSlaByClust)
  plt.legend(['Baseline'])